In [2]:
import rosbag
import numpy as np
from bisect import bisect_left
import rospy
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist, PoseStamped, PolygonStamped
from nav_msgs.msg import Path, Odometry
import csv
import os
import math
from tf_conversions import Quaternion

In [5]:

# Paths
bag_path = "/jackal_ws/src/mlda-barn-2024/inspection_data/2025-03-04-09-08-19.bag"
output_bag_path = "/jackal_ws/src/mlda-barn-2024/inspection_data/downsampled_rosbag.bag"
cmd_vel_topic = "/cmd_vel"
laser_scan_topic = "/laserscan"
odom_topic = '/odom'
goal_topic = "/move_base_simple/goal"
record_topic = "/matched_data"

#cmd_vel                 1039 msgs    : geometry_msgs/Twist  
# /laserscan                607 msgs    : sensor_msgs/LaserScan    
#      /move_base_simple/goal      1 msg     : geometry_msgs/PoseStamped
#      /odom                    1012 msgs    : nav_msgs/Odometry
# Store timestamps and messages
cmd_vel_msgs = []
laser_scan_msgs = []
odom_msgs = []

# Read the rosbag
with rosbag.Bag(bag_path, 'r') as bag:
    for topic, msg, t in bag.read_messages():
        timestamp = t.to_sec()
        
        if topic == cmd_vel_topic:
            cmd_vel_msgs.append((timestamp, msg))
        elif topic == laser_scan_topic:
            laser_scan_msgs.append((timestamp, msg))
            # print(topic,'  ',timestamp)
        elif topic == odom_topic:
            odom_msgs.append((timestamp,msg))
        elif topic == goal_topic:
            goal_x, goal_y = msg.pose.position.x, msg.pose.position.y
            print("goal: ", goal_x, goal_y)

            #trigger when new movebasegoal
                #if reach goal then save else no
        


('goal: ', 7.900001525878906, 12.649999618530273)


In [7]:
def processScan(data_dict, msg):
    assert(len(msg.ranges) == 720)
    cnt = 0
    for i in range(0, 720, 2):
        data_dict["lidar_" + str(cnt)] = min(5, msg.ranges[i])
        cnt += 1
    return data_dict


def processCmdVel(data_dict, msg):
    data_dict["cmd_vel_linear"] = msg.linear.x
    data_dict["cmd_vel_angular"] = msg.angular.z

    return data_dict

def euler_from_quaternion(x, y, z, w):
        # Roll (x-axis rotation)
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll = math.atan2(t0, t1)

        # Pitch (y-axis rotation)
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch = math.asin(t2)

        # Yaw (z-axis rotation)
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw = math.atan2(t3, t4)

        return roll, pitch, yaw


In [8]:
def processOdom(data_dict, msg):
    q = Quaternion()
    q.x = msg.pose.pose.orientation.x
    q.y = msg.pose.pose.orientation.y
    q.z = msg.pose.pose.orientation.z
    q.w = msg.pose.pose.orientation.w
    heading_rad = euler_from_quaternion(q.x, q.y, q.z, q.w)[2]

    data_dict["pos_x"] = msg.pose.pose.position.x
    data_dict["pos_y"] = msg.pose.pose.position.y
    data_dict["pose_heading"] = heading_rad
    data_dict["twist_linear"] = msg.twist.twist.linear.x
    data_dict["twist_angular"] = msg.twist.twist.angular.z

    return data_dict



# Extract timestamps as NumPy arrays for fast searching
cmd_vel_timestamps = np.array([t for t, _ in cmd_vel_msgs])
laser_scan_timestamps = np.array([t for t, _ in laser_scan_msgs])
odom_timestamps = np.array([t for t, _ in odom_msgs])

# Determine the start and end time
start_time = max(cmd_vel_timestamps[0], laser_scan_timestamps[0], odom_timestamps[0])  # Earliest common start
end_time = min(cmd_vel_timestamps[-1], laser_scan_timestamps[-1], odom_timestamps[-1])  # Latest common end

print("start:", cmd_vel_timestamps[0],' ',laser_scan_timestamps[0], odom_timestamps[0])
print("end:", cmd_vel_timestamps[-1],' ',laser_scan_timestamps[-1], odom_timestamps[-1])


# Generate 10Hz timestamps
downsampled_times = np.arange(start_time, end_time, 0.1)  # 10Hz = 0.1s interval

# Match closest messages to 10Hz timestamps
matched_data = []
average_timesteps_diff = []
data = []

file_exist = False

fieldnames = ["timestep"]
fieldnames += ['goal_x', 'goal_y']
fieldnames += ["cmd_vel_linear"]
fieldnames += ["cmd_vel_angular"] 
fieldnames += ["pos_x","pos_y", "pose_heading", "twist_linear","twist_angular"]
fieldnames += ["lidar_" + str(i) for i in range(360)]

file_path = 'test.csv'
if os.path.exists(file_path):
    file_exist = True
csv_file = open(file_path, 'a')
writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
if not file_exist:
    writer.writeheader()



for time in downsampled_times:
    data_dict = {}
    # Find closest cmd_vel
    idx = bisect_left(cmd_vel_timestamps, time)
    closest_cmd_idx = max(0, min(idx, len(cmd_vel_timestamps) - 1))
    closest_cmd_time, closest_cmd_msg = cmd_vel_msgs[closest_cmd_idx]
    # print("cmd:",closest_cmd_time )

    # Find closest laser_scan
    idx = bisect_left(laser_scan_timestamps, time)
    closest_laser_idx = max(0, min(idx, len(laser_scan_timestamps) - 1))
    closest_laser_time, closest_laser_msg = laser_scan_msgs[closest_laser_idx]
    # print("laser:",closest_laser_time )
    # print("----")

    idx = bisect_left(odom_timestamps, time)
    closest_odom_idx = max(0, min(idx, len(odom_timestamps) - 1))
    closest_odom_time, closest_odom_msg = odom_msgs[closest_odom_idx]
    # print("odom:",closest_odom_time )
    # print("----")
    average_timesteps_diff.append(abs(max(closest_cmd_time, closest_laser_time,closest_odom_time )-min(closest_cmd_time, closest_laser_time,closest_odom_time )) )

    
    data_dict = processScan(data_dict, closest_laser_msg)
    data_dict = processCmdVel(data_dict, closest_cmd_msg)
    data_dict = processOdom(data_dict, closest_odom_msg)
    
    #how to go next episode
    # print(len(data_dict.keys()))
    data.append(data_dict)
    matched_data.append((time, closest_cmd_msg, closest_odom_msg))

for i in range(len(data)):
    data[i]["timestep"] = i
    data[i]["goal_x"] = goal_x
    data[i]["goal_y"] = goal_y
    writer.writerow(data[i])
print(len(data))

print("average timesteps diff:", sum(average_timesteps_diff)/len(average_timesteps_diff))

# # Save to a new rosbag
# with rosbag.Bag(output_bag_path, 'w') as out_bag:
#     for time, cmd_msg, laser_msg in matched_data:
#         # print(laser_msg.data)
#         ros_time = rospy.Time.from_sec(time)
#         out_bag.write(record_topic + "/cmd_vel", cmd_msg, ros_time)
#         out_bag.write(record_topic + "/scan", laser_msg, ros_time)

# print("Saved downsampled data to" + output_bag_path)


('start:', 1266.847, ' ', 1266.8720000000001, 1266.8499999999999)
('end:', 1287.0799999999999, ' ', 1287.0730000000001, 1287.0709999999999)
202
('average timesteps diff:', 0.016019801980200785)
